In [0]:
combined_data = spark.read.parquet("/CombinedCleanedData/cleaned_combined.parquet")
combined_data.count()

Out[2]: 709262227

In [0]:
combined_data.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- color: string (nullable = true)



Business Question 1

* **What was the total number of trips?**
  

* **Which day of the week had the most trips?**
   

* **Which hour of the day had the most trips?**
   

* **What was the average number of passengers?**
   

* **What was the average amount paid per trip (using total_amount)?**
 

* **What was the average amount paid per passenger (using total_amount)?**
   


In [0]:
# Create a temporary view for the DataFrame to enable Spark SQL queries
combined_data.createOrReplaceTempView("trip_data")

# Query 1: Total number of trips for each year and month
total_trips_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        COUNT(*) AS total_trips
    FROM
        trip_data
    GROUP BY
        pickup_year, pickup_month
    ORDER BY
        pickup_year, pickup_month
"""

# Create a temporary view for Query 1 result
spark.sql(total_trips_query).createOrReplaceTempView("total_trips_result")

In [0]:
# Query 2: Day of the week with the most trips for each year and month
day_of_week_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        date_format(tpep_pickup_datetime, 'EEEE') AS day_of_week,
        COUNT(*) AS trip_count
    FROM
        trip_data
    GROUP BY
        pickup_year, pickup_month, day_of_week
    ORDER BY
        pickup_year, pickup_month, trip_count DESC
"""

# Create a temporary view for Query 2 result
spark.sql(day_of_week_query).createOrReplaceTempView("day_of_week_result")

In [0]:
# Query 3: Hour of the day with the most trips for each year and month
hour_of_day_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        HOUR(tpep_pickup_datetime) AS pickup_hour,
        COUNT(*) AS trip_count
    FROM
        trip_data
    GROUP BY
        pickup_year, pickup_month, pickup_hour
    ORDER BY
        pickup_year, pickup_month, trip_count DESC
"""

# Create a temporary view for Query 3 result
spark.sql(hour_of_day_query).createOrReplaceTempView("hour_of_day_result")


In [0]:
# Query 4: Average number of passengers for each year and month
avg_passengers_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        AVG(passenger_count) AS avg_passengers
    FROM
        trip_data
    GROUP BY
        pickup_year, pickup_month
"""

# Create a temporary view for Query 4 result
spark.sql(avg_passengers_query).createOrReplaceTempView("avg_passengers_result")


In [0]:
# Query 5: Average amount paid per trip for each year and month
avg_trip_amount_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        AVG(total_amount) AS avg_trip_amount
    FROM
        trip_data
    GROUP BY
        pickup_year, pickup_month
"""


In [0]:
# Create a temporary view for Query 5 result
spark.sql(avg_trip_amount_query).createOrReplaceTempView("avg_trip_amount_result")

# Query 6: Average amount paid per passenger for each year and month
avg_amount_per_passenger_query = """
    SELECT
        YEAR(tpep_pickup_datetime) AS pickup_year,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        AVG(total_amount / passenger_count) AS avg_amount_per_passenger
    FROM
        trip_data
    WHERE
        passenger_count > 0
    GROUP BY
        pickup_year, pickup_month
"""

# Create a temporary view for Query 6 result
spark.sql(avg_amount_per_passenger_query).createOrReplaceTempView("avg_amount_per_passenger_result")


In [0]:
# Join the results of all queries
combined_result_query = """
    SELECT
        t1.pickup_year,
        t1.pickup_month,
        t1.total_trips,
        t2.day_of_week,
        t3.pickup_hour, 
        t4.avg_passengers,
        t5.avg_trip_amount,
        t6.avg_amount_per_passenger
    FROM
        total_trips_result t1
    JOIN
        day_of_week_result t2
    ON
        t1.pickup_year = t2.pickup_year
        AND t1.pickup_month = t2.pickup_month
    JOIN
        hour_of_day_result t3
    ON
        t1.pickup_year = t3.pickup_year
        AND t1.pickup_month = t3.pickup_month
    JOIN
        avg_passengers_result t4
    ON
        t1.pickup_year = t4.pickup_year
        AND t1.pickup_month = t4.pickup_month
    JOIN
        avg_trip_amount_result t5
    ON
        t1.pickup_year = t5.pickup_year
        AND t1.pickup_month = t5.pickup_month
    JOIN
        avg_amount_per_passenger_result t6
    ON
        t1.pickup_year = t6.pickup_year
        AND t1.pickup_month = t6.pickup_month
"""

# Create a temporary view for the combined result
spark.sql(combined_result_query).createOrReplaceTempView("final_combined_result")

# Show the final combined result
final_combined_result = spark.sql("""
    SELECT
        pickup_year,
        pickup_month,
        MAX(total_trips) AS total_trips,
        MAX(day_of_week) AS day_of_week,
        MAX(pickup_hour) AS pickup_hour,
        MAX(avg_passengers) AS avg_passengers,
        MAX(avg_trip_amount) AS avg_trip_amount,
        MAX(avg_amount_per_passenger) AS avg_amount_per_passenger
    FROM
        final_combined_result
    GROUP BY
        pickup_year, pickup_month
    ORDER BY
        pickup_year, pickup_month
""")
final_combined_result.show()


+-----------+------------+-----------+-----------+-----------+------------------+------------------+------------------------+
|pickup_year|pickup_month|total_trips|day_of_week|pickup_hour|    avg_passengers|   avg_trip_amount|avg_amount_per_passenger|
+-----------+------------+-----------+-----------+-----------+------------------+------------------+------------------------+
|       2015|           1|   14086879|  Wednesday|         23|1.6537548168050566|14.757290035956748|      12.076119553604332|
|       2015|           2|   13856578|  Wednesday|         23|1.6406074429054562|15.234463717098825|       12.50917744970312|
|       2015|           3|   14885012|  Wednesday|         23|1.6409243069471493|15.658871816601016|      12.829251771698923|
|       2015|           4|   14557186|  Wednesday|         23| 1.646246808964315|15.848484455294047|      12.964614680993417|
|       2015|           5|   14768556|  Wednesday|         23|1.6528999179066661| 16.24036400454006|       13.22941284

In [0]:
final_combined_result.show(96)


+-----------+------------+-----------+-----------+-----------+------------------+------------------+------------------------+
|pickup_year|pickup_month|total_trips|day_of_week|pickup_hour|    avg_passengers|   avg_trip_amount|avg_amount_per_passenger|
+-----------+------------+-----------+-----------+-----------+------------------+------------------+------------------------+
|       2015|           1|   14086879|  Wednesday|         23|1.6537548168050566|14.757290035956748|      12.076119553604332|
|       2015|           2|   13856578|  Wednesday|         23|1.6406074429054562|15.234463717098825|       12.50917744970312|
|       2015|           3|   14885012|  Wednesday|         23|1.6409243069471493|15.658871816601016|      12.829251771698923|
|       2015|           4|   14557186|  Wednesday|         23| 1.646246808964315|15.848484455294047|      12.964614680993417|
|       2015|           5|   14768556|  Wednesday|         23|1.6528999179066661| 16.24036400454006|       13.22941284

Business Question 2

**For Yellow & Green Taxis:**
* **Average Trip Duration (minutes):**
  

* **Median Trip Duration (minutes):**


* **Minimum Trip Duration (minutes):**
 

* **Maximum Trip Duration (minutes):**


* **Average Trip Distance (km):**
  

* **Median Trip Distance (km):**
 

* **Minimum Trip Distance (km):**
 

* **Maximum Trip Distance (km):**


* **Average Speed (km/hour):**
 

* **Median Speed (km/hour):**
  

* **Minimum Speed (km/hour):**
  

* **Maximum Speed (km/hour):**
  


In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("TaxiAnalysis").getOrCreate()

# Create a temporary view for the DataFrame to enable Spark SQL queries
combined_data.createOrReplaceTempView("trip_data")

# Query 1: Average, Median, Minimum, and Maximum Trip Duration (in minutes) for Yellow Taxis
avg_trip_duration_query = """
    SELECT
        'Yellow' AS taxi_color,
        ROUND(AVG(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60), 2) AS avg_trip_duration_minutes,
        ROUND(percentile_approx(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60, 0.5), 2) AS median_trip_duration_minutes,
        MIN(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60) AS min_trip_duration_minutes,
        MAX(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60) AS max_trip_duration_minutes
    FROM
        trip_data
    WHERE
        color = 'yellow'
"""

# Create a temporary view for Query 1 result
spark.sql(avg_trip_duration_query).createOrReplaceTempView("avg_trip_duration_yellow")


In [0]:
# Query 2: Average, Median, Minimum, and Maximum Trip Duration (in minutes) for Green Taxis
avg_trip_duration_green_query = """
    SELECT
        'Green' AS taxi_color,
        ROUND(AVG(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60), 2) AS avg_trip_duration_minutes,
        ROUND(percentile_approx(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60, 0.5), 2) AS median_trip_duration_minutes,
        MIN(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60) AS min_trip_duration_minutes,
        MAX(TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 60) AS max_trip_duration_minutes
    FROM
        trip_data
    WHERE
        color = 'green'
"""

# Create a temporary view for Query 2 result
spark.sql(avg_trip_duration_green_query).createOrReplaceTempView("avg_trip_duration_green")


In [0]:
# Query 3: Average, Median, Minimum, and Maximum Trip Distance (in km) for Yellow Taxis
avg_trip_distance_yellow_query = """
    SELECT
        'Yellow' AS taxi_color,
        ROUND(AVG(trip_distance * 1.60934), 2) AS avg_trip_distance_km,
        ROUND(percentile_approx(trip_distance * 1.60934, 0.5), 2) AS median_trip_distance_km,
        MIN(trip_distance * 1.60934) AS min_trip_distance_km,
        MAX(trip_distance * 1.60934) AS max_trip_distance_km
    FROM
        trip_data
    WHERE
        color = 'yellow'
"""

# Create a temporary view for Query 3 result
spark.sql(avg_trip_distance_yellow_query).createOrReplaceTempView("avg_trip_distance_yellow")


In [0]:
# Query 4: Average, Median, Minimum, and Maximum Trip Distance (in km) for Green Taxis
avg_trip_distance_green_query = """
    SELECT
        'Green' AS taxi_color,
        ROUND(AVG(trip_distance * 1.60934), 2) AS avg_trip_distance_km,
        ROUND(percentile_approx(trip_distance * 1.60934, 0.5), 2) AS median_trip_distance_km,
        MIN(trip_distance * 1.60934) AS min_trip_distance_km,
        MAX(trip_distance * 1.60934) AS max_trip_distance_km
    FROM
        trip_data
    WHERE
        color = 'green'
"""

# Create a temporary view for Query 4 result
spark.sql(avg_trip_distance_green_query).createOrReplaceTempView("avg_trip_distance_green")


In [0]:
# Query 5: Average, Median, Minimum, and Maximum Speed (in km per hour) for Yellow Taxis
avg_speed_yellow_query = """
    SELECT
        'Yellow' AS taxi_color,
        ROUND(AVG((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)), 2) AS avg_speed_kph,
        ROUND(percentile_approx((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600), 0.5), 2) AS median_speed_kph,
        MIN((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)) AS min_speed_kph,
        MAX((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)) AS max_speed_kph
    FROM
        trip_data
    WHERE
        color = 'yellow'
"""

# Create a temporary view for Query 5 result
spark.sql(avg_speed_yellow_query).createOrReplaceTempView("avg_speed_yellow")


In [0]:
# Query 6: Average, Median, Minimum, and Maximum Speed (in km per hour) for Green Taxis
avg_speed_green_query = """
    SELECT
        'Green' AS taxi_color,
        ROUND(AVG((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)), 2) AS avg_speed_kph,
        ROUND(percentile_approx((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600), 0.5), 2) AS median_speed_kph,
        MIN((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)) AS min_speed_kph,
        MAX((trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600)) AS max_speed_kph
    FROM
        trip_data
    WHERE
        color = 'green'
"""

# Create a temporary view for Query 6 result
spark.sql(avg_speed_green_query).createOrReplaceTempView("avg_speed_green")


In [0]:
# Combine all the individual query results using UNION ALL
combined_query = """
    SELECT
        taxi_color,
        'Trip Duration (min)' AS metric_type,
        avg_trip_duration_minutes AS average,
        median_trip_duration_minutes AS median,
        min_trip_duration_minutes AS minimum,
        max_trip_duration_minutes AS maximum
    FROM
        avg_trip_duration_yellow
    UNION ALL
    SELECT
        taxi_color,
        'Trip Duration (min)' AS metric_type,
        avg_trip_duration_minutes AS average,
        median_trip_duration_minutes AS median,
        min_trip_duration_minutes AS minimum,
        max_trip_duration_minutes AS maximum
    FROM
        avg_trip_duration_green
    UNION ALL
    SELECT
        taxi_color,
        'Trip Distance (km)' AS metric_type,
        avg_trip_distance_km AS average,
        median_trip_distance_km AS median,
        min_trip_distance_km AS minimum,
        max_trip_distance_km AS maximum
    FROM
        avg_trip_distance_yellow
    UNION ALL
    SELECT
        taxi_color,
        'Trip Distance (km)' AS metric_type,
        avg_trip_distance_km AS average,
        median_trip_distance_km AS median,
        min_trip_distance_km AS minimum,
        max_trip_distance_km AS maximum
    FROM
        avg_trip_distance_green
    UNION ALL
    SELECT
        taxi_color,
        'Speed (km/h)' AS metric_type,
        avg_speed_kph AS average,
        median_speed_kph AS median,
        min_speed_kph AS minimum,
        max_speed_kph AS maximum
    FROM
        avg_speed_yellow
    UNION ALL
    SELECT
        taxi_color,
        'Speed (km/h)' AS metric_type,
        avg_speed_kph AS average,
        median_speed_kph AS median,
        min_speed_kph AS minimum,
        max_speed_kph AS maximum
    FROM
        avg_speed_green
"""

# Create a temporary view for the combined result
spark.sql(combined_query).createOrReplaceTempView("combined_result")

# Retrieve the final combined result
final_combined_result = spark.sql("SELECT * FROM combined_result ORDER BY taxi_color, metric_type")

# Show the final combined result
final_combined_result.show()


+----------+-------------------+-------+------+--------------------+------------------+
|taxi_color|        metric_type|average|median|             minimum|           maximum|
+----------+-------------------+-------+------+--------------------+------------------+
|     Green|       Speed (km/h)|  20.03| 18.26|0.006710010076091867|103.59571506531205|
|     Green| Trip Distance (km)|   4.75|  3.06| 0.16093400000000002|       160.8857198|
|     Green|Trip Duration (min)|  20.24| 10.57|                 1.0|1439.9833333333333|
|    Yellow|       Speed (km/h)|  18.67| 16.41|0.006706281904365039| 103.5978528813559|
|    Yellow| Trip Distance (km)|   4.85|  2.74| 0.16093400000000002|       160.8696264|
|    Yellow|Trip Duration (min)|  16.71| 11.27|                 1.0|1439.9833333333333|
+----------+-------------------+-------+------+--------------------+------------------+



Business Question 3

**For each combination of Taxi Color, Pick-Up, Drop-Off, Month, Day of the Week, and Hour:**

* **Total Number of Trips:**


* **Average Distance (km):**


* **Average Amount Paid per Trip (using total_amount):**
 

* **Total Amount Paid (using total_amount):**



In [0]:
from pyspark.sql import SparkSession

merged_data = spark.read.parquet("/MergedData/merged_both_locations.parquet")
merged_data.count()

Out[1]: 605365170


**Important Note:** It's crucial to understand that the 'merged_data' we're working with isn't a complete dataset due to the challenges we faced earlier. As a result, the answers to the following business queries may not be comprehensive:

**For each combination of Taxi Color, Pick-Up, Drop-Off, Month, Day of the Week, and Hour:**

- Total Number of Trips:
- Average Distance (in kilometers):
- Average Amount Paid per Trip (calculated using the 'total_amount' field):
- Total Amount Paid (calculated using the 'total_amount' field):

Please do keep in mind the limitations we encountered when interpreting the results, as they may not represent the entire dataset.


In [0]:
merged_data.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- Combined_PULocationID: long (nullable = true)
 |-- Combined_DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- color: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- PULocation_Borough: string (nullab

In [0]:

merged_data.createOrReplaceTempView("location_trip_data")

# Create a Spark session
spark = SparkSession.builder.appName("TaxiDataAnalysis").getOrCreate()

# Your initial query to calculate total trips
total_location_trips_query = """
    SELECT
        color AS taxi_color,
        PULocation_Borough AS pickup_location,
        DOLocation_Borough AS dropoff_location,
        MONTH(tpep_pickup_datetime) AS pickup_month,
        date_format(tpep_pickup_datetime, 'EEEE') AS day_of_week,
        HOUR(tpep_pickup_datetime) AS pickup_hour,
        COUNT(*) AS total_trips,
        AVG(trip_distance) AS avg_distance,
        AVG(total_amount) AS avg_amount_per_trip,
        SUM(total_amount) AS total_amount_paid
    FROM
        location_trip_data
    GROUP BY
        color, PULocation_Borough, DOLocation_Borough , pickup_month, day_of_week, pickup_hour
"""

# Execute the query and store the result in a DataFrame
location_results_df = spark.sql(total_location_trips_query)

# Show the results
location_results_df.show()

+----------+---------------+----------------+------------+-----------+-----------+-----------+------------------+-------------------+------------------+
|taxi_color|pickup_location|dropoff_location|pickup_month|day_of_week|pickup_hour|total_trips|      avg_distance|avg_amount_per_trip| total_amount_paid|
+----------+---------------+----------------+------------+-----------+-----------+-----------+------------------+-------------------+------------------+
|    yellow|      Manhattan|       Manhattan|           2|   Saturday|          7|      93659|1.9497606209760943| 11.017263050001253|1031865.8400000673|
|    yellow|         Queens|          Queens|           2|   Saturday|         10|       4393| 4.622219440018211|  17.89408149328481| 78608.70000000016|
|    yellow|        Unknown|           Bronx|           2|   Saturday|         21|          9|12.147777777777778|  43.32444444444445| 389.9200000000001|
|    yellow|       Brooklyn|           Bronx|           2|     Sunday|          2|

In [0]:
from pyspark.sql.functions import avg

average_avg_amount_per_trip = location_results_df.agg(avg("avg_amount_per_trip")).collect()[0][0]

print("Average Avg Amount Per Trip:", average_avg_amount_per_trip)


Average Avg Amount Per Trip: 43.47990516867878


In [0]:
location_results_df.count()

Business Question 4

**The percentage of trips where drivers received tips:**


In [0]:
# Calculate the percentage of trips where drivers received tips
percentage_tip_trips_query = """
    SELECT
        (COUNT(CASE WHEN tip_amount > 0 THEN 1 ELSE NULL END) / COUNT(*)) * 100 AS percentage_tip_trips
    FROM
        trip_data
"""

# Execute the query
percentage_tip_trips = spark.sql(percentage_tip_trips_query)

# Show the percentage of trips with tips
percentage_tip_trips.show()


+--------------------+
|percentage_tip_trips|
+--------------------+
|   63.57964160975966|
+--------------------+



Business Question 5

**For trips where the driver received tips, the percentage where the driver received tips of at least $5:**

In [0]:
# Calculate the percentage of trips where the driver received tips of at least $5 among tip-receiving trips
percentage_high_tip_trips_query = """
    SELECT
        (COUNT(CASE WHEN tip_amount >= 5.0 THEN 1 ELSE NULL END) / COUNT(CASE WHEN tip_amount > 0 THEN 1 ELSE NULL END)) * 100 AS percentage_high_tip_trips
    FROM
        trip_data
    WHERE
        tip_amount > 0
"""

# Execute the query
percentage_high_tip_trips = spark.sql(percentage_high_tip_trips_query)

# Show the percentage of trips with tips of at least $5 among tip-receiving trips
percentage_high_tip_trips.show()


+-------------------------+
|percentage_high_tip_trips|
+-------------------------+
|        12.19731329388956|
+-------------------------+



Business Question 6

**For trips with a duration under 5 mins:**
* **Average Speed (km per hour):**
  

* **Average Distance per Dollar (km per $):**


**For trips from 5 mins to 10 mins:**
* **Average Speed (km per hour):**


* **Average Distance per Dollar (km per $):**
 

**For trips from 10 mins to 20 mins:**
* **Average Speed (km per hour):**


* **Average Distance per Dollar (km per $):**
  

**For trips from 20 mins to 30 mins:**
* **Average Speed (km per hour):**


* **Average Distance per Dollar (km per $):**
  

**For trips from 30 mins to 60 mins:**
* **Average Speed (km per hour):**
 

* **Average Distance per Dollar (km per $):**
 

**For trips of at least 60 mins:**
* **Average Speed (km per hour):**
  

* **Average Distance per Dollar (km per $):**



In [0]:
# Classify trips into duration bins and calculate average speed and average distance per dollar
duration_bins_query = """
    SELECT
        CASE
            WHEN duration_hours < 5 THEN 'Under 5 Mins'
            WHEN duration_hours >= 5 AND duration_hours < 10 THEN '5 to 10 Mins'
            WHEN duration_hours >= 10 AND duration_hours < 20 THEN '10 to 20 Mins'
            WHEN duration_hours >= 20 AND duration_hours < 30 THEN '20 to 30 Mins'
            WHEN duration_hours >= 30 AND duration_hours < 60 THEN '30 to 60 Mins'
            ELSE 'At least 60 Mins'
        END AS duration_bin,
        AVG(speed_kph) AS average_speed_kph,
        AVG((trip_distance * 1.60934) / total_amount) AS average_distance_per_dollar_km_per_dollar
    FROM (
        SELECT
            trip_data.*,
            TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600 AS duration_hours,
            (trip_distance * 1.60934) / (TIMESTAMPDIFF(SECOND, tpep_pickup_datetime, tpep_dropoff_datetime) / 3600) AS speed_kph
        FROM
            trip_data
    ) AS trip_stats
    GROUP BY
        duration_bin
    ORDER BY
        duration_bin
"""


# Execute the corrected query
duration_bins_result = spark.sql(duration_bins_query)

# Show the result with duration bins, average speed, and average distance per dollar
duration_bins_result.show()


+-------------+-------------------+-----------------------------------------+
| duration_bin|  average_speed_kph|average_distance_per_dollar_km_per_dollar|
+-------------+-------------------+-----------------------------------------+
|10 to 20 Mins| 0.6931934497961179|                      0.34070894858536965|
|20 to 30 Mins|0.25112726626824255|                      0.27014632764178487|
| 5 to 10 Mins| 1.5317699483554243|                       1.0218401828199113|
| Under 5 Mins| 18.827787877018018|                      0.24709248189093522|
+-------------+-------------------+-----------------------------------------+

